# Start notebook 

In [7]:
# load libraries
import statsmodels.stats.multicomp
from statsmodels.formula.api import ols
import statsmodels.api as sm
import researchpy as rp
import statsFuncs.mean_confidence_interval as ms
import plotFuncs
import scipy
from scipy import stats as stats
from scipy import stats as cp
import os
from scipy.io import loadmat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as matplotlib
import seaborn as sns
import glob as glob
from natsort import natsorted
sns.set(color_codes=True)
from statsmodels.stats.anova import AnovaRM
import pingouin as pg

# stats
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
import scikit_posthocs as sp

# magic functions
%load_ext autoreload
%autoreload 2
%matplotlib inline

# create dummy class for struct-like dataframes


class structtype():
    pass


matplotlib.rc('xtick', labelsize=20)
matplotlib.rc('ytick', labelsize=20)
matplotlib.rcParams.update({'axes.labelsize': 16.0})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Paths 

In [8]:
analysispath = os.getcwd()
mainpath = analysispath + '/../'
designpath = mainpath + '/design'
resultspath = mainpath + '/results'
stimpath = mainpath + '/stim'
practicestimpath = mainpath + '/practice'
results_csv = mainpath + '/results_csv'
analysispath = mainpath + '/analysis'
design_csv = mainpath + '/design_csv'

# Load data 

In [9]:
allsubjects = pd.read_csv('allsubjects_perifovmooney_ecc_post.csv', index_col=0)

# Separate data by upright or inverted 

In [10]:
upright_data = allsubjects[allsubjects['COND'] == 'upright']
inverted_data = allsubjects[allsubjects['COND'] == 'inverted']


# Holistic 

## absolute eccentricity 

In [67]:
numLocs = len(allsubjects.ECC_PLOT_ABS.unique())
numCategories = len(allsubjects.MOONEYCATEGORY.unique())
numSubs  = len(allsubjects.SUBINIT.unique())
accPerSubLocs = pd.DataFrame()

categorynames = list(allsubjects.MOONEYCATEGORY.unique())
locationnames = list(allsubjects.ECC_PLOT_ABS.unique())
subinitnames = list(allsubjects.SUBINIT.unique())

row = 0
for subject in subinitnames:
    # index by subject
    subdata = allsubjects[allsubjects['SUBINIT'] == subject]
    
    for location in locationnames:
        # index by location
        locationdata = subdata[subdata['ECC_PLOT_ABS'] == location]
        
        for category in categorynames:
            # index by category
                
            categorydata = locationdata[locationdata['MOONEYCATEGORY'] == category]

            # index the datapoints from each locations
            indexed_upright = categorydata[categorydata['COND'] == 'upright']

            indexed_inverted = categorydata[categorydata['COND'] == 'inverted']
            

            m_upright, lower_upright, upper_upright = ms.mean_confidence_interval(
                indexed_upright.loc[:, 'ACCURACY'])
            m_inverted, lower_inverted, upper_inverted = ms.mean_confidence_interval(
                indexed_inverted.loc[:, 'ACCURACY'])

            accPerSubLocs.loc[row, 'SUBINIT'] = subject
            accPerSubLocs.loc[row, 'ECC_PLOT_ABS'] = location
            accPerSubLocs.loc[row, 'MOONEYCATEGORY'] = category
            accPerSubLocs.loc[row, 'MEAN_UPRIGHT'] = m_upright
            accPerSubLocs.loc[row, 'CI_LOWER_UPRIGHT'] = lower_upright
            accPerSubLocs.loc[row, 'CI_UPPER_UPRIGHT'] = upper_upright

            accPerSubLocs.loc[row, 'MEAN_INVERTED'] = m_inverted
            accPerSubLocs.loc[row, 'CI_LOWER_INVERTED'] = lower_inverted
            accPerSubLocs.loc[row, 'CI_UPPER_INVERTED'] = upper_inverted

            accPerSubLocs.loc[row, 'HOLISTIC'] = accPerSubLocs.loc[row,
                                                              'MEAN_UPRIGHT'] - accPerSubLocs.loc[row, 'MEAN_INVERTED']
            row = row + 1

In [68]:
# Fits the model with the interaction term
# This will also automatically include the main effects for each factor
model = ols('HOLISTIC ~ C(MOONEYCATEGORY)*C(ECC_PLOT_ABS)', accPerSubLocs).fit()

# Seeing if the overall model is significant
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")
if model.f_pvalue < 0.05:
    print('The overall model is significant')
    
    
print(rp.summary_cont(accPerSubLocs.groupby(['MOONEYCATEGORY', 'ECC_PLOT_ABS']))['HOLISTIC'])
print(model.summary())


# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
print(res)
print('\n\nThe interaction is not significant p = 0.1')

mc = statsmodels.stats.multicomp.MultiComparison(accPerSubLocs['HOLISTIC'], accPerSubLocs['ECC_PLOT_ABS'])
mc_results = mc.tukeyhsd()
print(mc_results)

Overall model F( 13, 336) =  13.352, p =  0.0000
The overall model is significant


                              N    Mean        SD        SE  95% Conf.  \
MOONEYCATEGORY ECC_PLOT_ABS                                              
high holistic  0.0           25  0.2560  0.134164  0.026833   0.203408   
               1.0           25  0.2360  0.137235  0.027447   0.182204   
               2.0           25  0.2264  0.100452  0.020090   0.187023   
               3.0           25  0.2160  0.104722  0.020944   0.174949   
               4.0           25  0.2144  0.145776  0.029155   0.157256   
               5.0           25  0.1824  0.144028  0.028806   0.125941   
               6.0           25  0.1816  0.117604  0.023521   0.135499   
low holistic   0.0           25  0.0544  0.066212  0.013242   0.028445   
               1.0           25  0.0528  0.078077  0.015615   0.022194   
               2.0           25  0.0664  0.074771  0.014954   0.037090   
               3.0          

/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:151: RuntimeWarning: invalid value encountered in sqrt
  l_ci = lambda x: numpy.mean(x) - (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))
/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:154: RuntimeWarning: invalid value encountered in sqrt
  u_ci = lambda x: numpy.mean(x) + (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))


### Repeated measures

In [69]:
aovrm2way = AnovaRM(accPerSubLocs, 'HOLISTIC', 'SUBINIT', within=['ECC_PLOT_ABS', 'MOONEYCATEGORY'])
res2way = aovrm2way.fit()

print(res2way)

                          Anova
                            F Value Num DF  Den DF  Pr > F
----------------------------------------------------------
ECC_PLOT_ABS                 0.3024 6.0000 144.0000 0.9348
MOONEYCATEGORY              68.5187 1.0000  24.0000 0.0000
ECC_PLOT_ABS:MOONEYCATEGORY  2.9421 6.0000 144.0000 0.0097



In [70]:
aov = pg.rm_anova(data=accPerSubLocs, dv='HOLISTIC', within=['MOONEYCATEGORY', 'ECC_PLOT_ABS'], subject='SUBINIT', detailed=True, effsize='n2')
pg.print_table(aov)
x = pg.pairwise_ttests(data=accPerSubLocs, dv='HOLISTIC', within=[ 'MOONEYCATEGORY', 'ECC_PLOT_ABS'], subject='SUBINIT', marginal=True, padjust='holm', effsize= 'eta-square')
x[x['Contrast'] == 'MOONEYCATEGORY * ECC_PLOT_ABS'].to_csv('ttests_abseccs.csv')


ANOVA SUMMARY

Source                            SS    ddof1    ddof2     MS       F    p-unc    p-GG-corr     n2    eps
-----------------------------  -----  -------  -------  -----  ------  -------  -----------  -----  -----
MOONEYCATEGORY                 1.780        1       24  1.780  68.519    0.000        0.000  0.364  1.000
ECC_PLOT_ABS                   0.014        6      144  0.002   0.302    0.935        0.915  0.003  0.861
MOONEYCATEGORY * ECC_PLOT_ABS  0.145        6      144  0.024   2.942    0.010        0.013  0.030  0.882



In [71]:
x

,Contrast,MOONEYCATEGORY,A,B,Paired,Parametric,T,dof,Tail,p-unc,p-corr,p-adjust,BF10,eta-square
0,MOONEYCATEGORY,-,high holistic,low holistic,True,True,8.277602,24.0,two-sided,1.716419e-08,NaN,NaN,7.778e+05,0.544672
1,ECC_PLOT_ABS,-,6,2,True,True,-0.343935,24.0,two-sided,7.338902e-01,1.0,holm,0.223,0.002237
2,ECC_PLOT_ABS,-,6,1,True,True,-0.198435,24.0,two-sided,8.443759e-01,1.0,holm,0.215,0.000645
3,ECC_PLOT_ABS,-,6,3,True,True,-0.615509,24.0,two-sided,5.440122e-01,1.0,holm,0.251,0.006932
4,ECC_PLOT_ABS,-,6,0,True,True,-0.815901,24.0,two-sided,4.225811e-01,1.0,holm,0.285,0.010391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,MOONEYCATEGORY * ECC_PLOT_ABS,low holistic,3,5,True,True,-0.035426,24.0,two-sided,9.720334e-01,1.0,holm,0.211,0.000023
60,MOONEYCATEGORY * ECC_PLOT_ABS,low holistic,3,4,True,True,0.826961,24.0,two-sided,4.164058e-01,1.0,holm,0.287,0.014386
61,MOONEYCATEGORY * ECC_PLOT_ABS,low holistic,0,5,True,True,-1.481610,24.0,two-sided,1.514555e-01,1.0,holm,0.554,0.052355
62,MOONEYCATEGORY * ECC_PLOT_ABS,low holistic,0,4,True,True,-0.556287,24.0,two-sided,5.831645e-01,1.0,holm,0.243,0.006597


## all eccentricites 

In [54]:
numLocs = len(allsubjects.ECC_PLOT.unique())
numCategories = len(allsubjects.MOONEYCATEGORY.unique())
numSubs  = len(allsubjects.SUBINIT.unique())
accPerSubLocs = pd.DataFrame()

categorynames = list(allsubjects.MOONEYCATEGORY.unique())
locationnames = list(allsubjects.ECC_PLOT.unique())
subinitnames = list(allsubjects.SUBINIT.unique())

row = 0
for subject in subinitnames:
    # index by subject
    subdata = allsubjects[allsubjects['SUBINIT'] == subject]
    
    for location in locationnames:
        # index by location
        locationdata = subdata[subdata['ECC_PLOT'] == location]
        
        for category in categorynames:
            # index by category
                
            categorydata = locationdata[locationdata['MOONEYCATEGORY'] == category]

            # index the datapoints from each locations
            indexed_upright = categorydata[categorydata['COND'] == 'upright']

            indexed_inverted = categorydata[categorydata['COND'] == 'inverted']
            

            m_upright, lower_upright, upper_upright = ms.mean_confidence_interval(
                indexed_upright.loc[:, 'ACCURACY'])
            m_inverted, lower_inverted, upper_inverted = ms.mean_confidence_interval(
                indexed_inverted.loc[:, 'ACCURACY'])

            accPerSubLocs.loc[row, 'SUBINIT'] = subject
            accPerSubLocs.loc[row, 'ECC_PLOT'] = location
            accPerSubLocs.loc[row, 'MOONEYCATEGORY'] = category
            accPerSubLocs.loc[row, 'MEAN_UPRIGHT'] = m_upright
            accPerSubLocs.loc[row, 'CI_LOWER_UPRIGHT'] = lower_upright
            accPerSubLocs.loc[row, 'CI_UPPER_UPRIGHT'] = upper_upright

            accPerSubLocs.loc[row, 'MEAN_INVERTED'] = m_inverted
            accPerSubLocs.loc[row, 'CI_LOWER_INVERTED'] = lower_inverted
            accPerSubLocs.loc[row, 'CI_UPPER_INVERTED'] = upper_inverted

            accPerSubLocs.loc[row, 'HOLISTIC'] = accPerSubLocs.loc[row,
                                                              'MEAN_UPRIGHT'] - accPerSubLocs.loc[row, 'MEAN_INVERTED']
            row = row + 1

In [55]:
# Fits the model with the interaction term
# This will also automatically include the main effects for each factor
model = ols('HOLISTIC ~ C(MOONEYCATEGORY)*C(ECC_PLOT)', accPerSubLocs).fit()

# Seeing if the overall model is significant
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")
if model.f_pvalue < 0.05:
    print('The overall model is significant')
    
    
print(rp.summary_cont(accPerSubLocs.groupby(['MOONEYCATEGORY', 'ECC_PLOT']))['HOLISTIC'])
print(model.summary())


# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
print(res)
print('\n\nThe interaction is not significant p = 0.1')

mc = statsmodels.stats.multicomp.MultiComparison(accPerSubLocs['HOLISTIC'], accPerSubLocs['ECC_PLOT'])
mc_results = mc.tukeyhsd()
print(mc_results)

Overall model F( 25, 624) =  7.923, p =  0.0000
The overall model is significant


                          N    Mean        SD        SE  95% Conf.  Interval
MOONEYCATEGORY ECC_PLOT                                                     
high holistic  -6.0      25  0.1872  0.127917  0.025583   0.137057  0.237343
               -5.0      25  0.2064  0.183550  0.036710   0.134448  0.278352
               -4.0      25  0.2064  0.191025  0.038205   0.131518  0.281282
               -3.0      25  0.2240  0.124900  0.024980   0.175039  0.272961
               -2.0      25  0.2544  0.130578  0.026116   0.203213  0.305587
               -1.0      25  0.2320  0.173205  0.034641   0.164104  0.299896
                0.0      25  0.2560  0.134164  0.026833   0.203408  0.308592
                1.0      25  0.2400  0.180370  0.036074   0.169295  0.310705
                2.0      25  0.1984  0.131899  0.026380   0.146696  0.250104
                3.0      25  0.2080  0.154488  0.030898   0.147441  0.

/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:151: RuntimeWarning: invalid value encountered in sqrt
  l_ci = lambda x: numpy.mean(x) - (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))
/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:154: RuntimeWarning: invalid value encountered in sqrt
  u_ci = lambda x: numpy.mean(x) + (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))


Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
  -6.0   -5.0   0.0256    0.9 -0.0752 0.1264  False
  -6.0   -4.0   0.0184    0.9 -0.0824 0.1192  False
  -6.0   -3.0   0.0256    0.9 -0.0752 0.1264  False
  -6.0   -2.0   0.0416    0.9 -0.0592 0.1424  False
  -6.0   -1.0   0.0184    0.9 -0.0824 0.1192  False
  -6.0    0.0    0.024    0.9 -0.0768 0.1248  False
  -6.0    1.0    0.008    0.9 -0.0928 0.1088  False
  -6.0    2.0  -0.0112    0.9  -0.112 0.0896  False
  -6.0    3.0   0.0152    0.9 -0.0856  0.116  False
  -6.0    4.0     -0.0    0.9 -0.1008 0.1008  False
  -6.0    5.0  -0.0176    0.9 -0.1184 0.0832  False
  -6.0    6.0   0.0184    0.9 -0.0824 0.1192  False
  -5.0   -4.0  -0.0072    0.9  -0.108 0.0936  False
  -5.0   -3.0     -0.0    0.9 -0.1008 0.1008  False
  -5.0   -2.0    0.016    0.9 -0.0848 0.1168  False
  -5.0   -1.0  -0.0072    0.9  -0.108 0.0936  False
  -5.0    0.

In [56]:
aovrm2way = AnovaRM(accPerSubLocs, 'HOLISTIC', 'SUBINIT', within=['ECC_PLOT', 'MOONEYCATEGORY'])
res2way = aovrm2way.fit()

print(res2way)

                         Anova
                        F Value  Num DF  Den DF  Pr > F
-------------------------------------------------------
ECC_PLOT                 0.9038 12.0000 288.0000 0.5435
MOONEYCATEGORY          59.9034  1.0000  24.0000 0.0000
ECC_PLOT:MOONEYCATEGORY  1.7191 12.0000 288.0000 0.0622



In [61]:
aov = pg.rm_anova(data=accPerSubLocs, dv='HOLISTIC', within=['MOONEYCATEGORY', 'ECC_PLOT'], subject='SUBINIT', detailed=True, effsize='n2')
pg.print_table(aov)
x = pg.pairwise_ttests(data=accPerSubLocs, dv='HOLISTIC', within=[ 'MOONEYCATEGORY', 'ECC_PLOT'], subject='SUBINIT', marginal=True, padjust='holm', effsize= 'eta-square')


ANOVA SUMMARY

Source                        SS    ddof1    ddof2     MS       F    p-unc    p-GG-corr     n2    eps
-------------------------  -----  -------  -------  -----  ------  -------  -----------  -----  -----
MOONEYCATEGORY             3.099        1       24  3.099  59.903    0.000        0.000  0.232  1.000
ECC_PLOT                   0.162       12      288  0.013   0.904    0.544        0.517  0.012  0.695
MOONEYCATEGORY * ECC_PLOT  0.305       12      288  0.025   1.719    0.062        0.091  0.023  0.705



,Contrast,MOONEYCATEGORY,A,B,Paired,Parametric,T,dof,Tail,p-unc,p-corr,p-adjust,BF10,eta-square
0,MOONEYCATEGORY,-,high holistic,low holistic,True,True,7.739726,24.0,two-sided,5.638821e-08,NaN,NaN,2.566e+05,0.525643
1,ECC_PLOT,-,-6,2,True,True,0.399294,24.0,two-sided,6.932073e-01,1.0,holm,0.227,0.003970
2,ECC_PLOT,-,-6,1,True,True,-0.291214,24.0,two-sided,7.733896e-01,1.0,holm,0.219,0.001489
3,ECC_PLOT,-,-6,-3,True,True,-1.119025,24.0,two-sided,2.742048e-01,1.0,holm,0.369,0.023826
4,ECC_PLOT,-,-6,0,True,True,-0.952661,24.0,two-sided,3.502587e-01,1.0,holm,0.317,0.019286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,MOONEYCATEGORY * ECC_PLOT,low holistic,-5,-1,True,True,1.348400,24.0,two-sided,1.901183e-01,1.0,holm,0.472,0.038600
231,MOONEYCATEGORY * ECC_PLOT,low holistic,-5,3,True,True,0.745938,24.0,two-sided,4.629493e-01,1.0,holm,0.271,0.009182
232,MOONEYCATEGORY * ECC_PLOT,low holistic,-2,-1,True,True,0.911465,24.0,two-sided,3.711149e-01,1.0,holm,0.307,0.014173
233,MOONEYCATEGORY * ECC_PLOT,low holistic,-2,3,True,True,0.190476,24.0,two-sided,8.505380e-01,1.0,holm,0.214,0.000753


In [65]:
x[x['Contrast'] == 'MOONEYCATEGORY * ECC_PLOT'].to_csv('ttests_alleccs.csv')

In [60]:
pg.print_table(x)


POST HOC TESTS

Contrast                   MOONEYCATEGORY    A              B             Paired    Parametric         T     dof  Tail         p-unc    p-corr  p-adjust          BF10    eta-square
-------------------------  ----------------  -------------  ------------  --------  ------------  ------  ------  ---------  -------  --------  ----------  ----------  ------------
MOONEYCATEGORY             -                 high holistic  low holistic  True      True           7.740  24.000  two-sided    0.000   nan      nan         256600.000         0.526
ECC_PLOT                   -                 -6.0           2.0           True      True           0.399  24.000  two-sided    0.693     1.000  holm             0.227         0.004
ECC_PLOT                   -                 -6.0           1.0           True      True          -0.291  24.000  two-sided    0.773     1.000  holm             0.219         0.001
ECC_PLOT                   -                 -6.0           -3.0          True

## Left vs Right 

In [17]:
numLocs = len(allsubjects.LOC.unique())
numCategories = len(allsubjects.MOONEYCATEGORY.unique())
numSubs  = len(allsubjects.SUBINIT.unique())
accPerSubLocs = pd.DataFrame()

categorynames = list(allsubjects.MOONEYCATEGORY.unique())
locationnames = list(allsubjects.LOC.unique())
subinitnames = list(allsubjects.SUBINIT.unique())

row = 0
for subject in subinitnames:
    # index by subject
    subdata = allsubjects[allsubjects['SUBINIT'] == subject]
    
    for location in locationnames:
        # index by location
        locationdata = subdata[subdata['LOC'] == location]
        
        for category in categorynames:
            # index by category
                
            categorydata = locationdata[locationdata['MOONEYCATEGORY'] == category]

            # index the datapoints from each locations
            indexed_upright = categorydata[categorydata['COND'] == 'upright']

            indexed_inverted = categorydata[categorydata['COND'] == 'inverted']
            

            m_upright, lower_upright, upper_upright = ms.mean_confidence_interval(
                indexed_upright.loc[:, 'ACCURACY'])
            m_inverted, lower_inverted, upper_inverted = ms.mean_confidence_interval(
                indexed_inverted.loc[:, 'ACCURACY'])

            accPerSubLocs.loc[row, 'SUBINIT'] = subject
            accPerSubLocs.loc[row, 'LOC'] = location
            accPerSubLocs.loc[row, 'MOONEYCATEGORY'] = category
            accPerSubLocs.loc[row, 'MEAN_UPRIGHT'] = m_upright
            accPerSubLocs.loc[row, 'CI_LOWER_UPRIGHT'] = lower_upright
            accPerSubLocs.loc[row, 'CI_UPPER_UPRIGHT'] = upper_upright

            accPerSubLocs.loc[row, 'MEAN_INVERTED'] = m_inverted
            accPerSubLocs.loc[row, 'CI_LOWER_INVERTED'] = lower_inverted
            accPerSubLocs.loc[row, 'CI_UPPER_INVERTED'] = upper_inverted

            accPerSubLocs.loc[row, 'HOLISTIC'] = accPerSubLocs.loc[row,
                                                              'MEAN_UPRIGHT'] - accPerSubLocs.loc[row, 'MEAN_INVERTED']
            row = row + 1

In [18]:
# Fits the model with the interaction term
# This will also automatically include the main effects for each factor
model = ols('HOLISTIC ~ C(MOONEYCATEGORY)*C(LOC)', accPerSubLocs).fit()

# Seeing if the overall model is significant
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")
if model.f_pvalue < 0.05:
    print('The overall model is significant')
    
    
print(rp.summary_cont(accPerSubLocs.groupby(['MOONEYCATEGORY', 'LOC']))['HOLISTIC'])
print(model.summary())


# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
print(res)
print('\n\nThe interaction is not significant p = 0.1')

mc = statsmodels.stats.multicomp.MultiComparison(accPerSubLocs['HOLISTIC'], accPerSubLocs['LOC'])
mc_results = mc.tukeyhsd()
print(mc_results)

Overall model F( 5, 144) =  25.381, p =  0.0000
The overall model is significant


                       N      Mean        SD        SE  95% Conf.  Interval
MOONEYCATEGORY LOC                                                         
high holistic  fovea  25  0.256000  0.134164  0.026833   0.203408  0.308592
               left   25  0.218400  0.102022  0.020404   0.178407  0.258393
               right  25  0.200533  0.086258  0.017252   0.166720  0.234346
low holistic   fovea  25  0.054400  0.066212  0.013242   0.028445  0.080355
               left   25  0.087200  0.052596  0.010519   0.066582  0.107818
               right  25  0.066133  0.050476  0.010095   0.046347  0.085920
                            OLS Regression Results                            
Dep. Variable:               HOLISTIC   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.450
Method:                 Least Squares   F-statistic:                    

/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:151: RuntimeWarning: invalid value encountered in sqrt
  l_ci = lambda x: numpy.mean(x) - (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))
/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:154: RuntimeWarning: invalid value encountered in sqrt
  u_ci = lambda x: numpy.mean(x) + (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))


### Repeated measures 

In [19]:
aovrm2way = AnovaRM(accPerSubLocs, 'HOLISTIC', 'SUBINIT', within=['LOC', 'MOONEYCATEGORY'])
res2way = aovrm2way.fit()

print(res2way)

                     Anova
                   F Value Num DF  Den DF Pr > F
------------------------------------------------
LOC                 1.9163 2.0000 48.0000 0.1582
MOONEYCATEGORY     87.2074 1.0000 24.0000 0.0000
LOC:MOONEYCATEGORY  3.7279 2.0000 48.0000 0.0313



In [45]:
import pingouin as pg
aov = pg.rm_anova(data=accPerSubLocs, dv='HOLISTIC', within=['LOC', 'MOONEYCATEGORY'], subject='SUBINIT', detailed=True, effsize='n2')
pg.print_table(aov)


ANOVA SUMMARY

Source                   SS    ddof1    ddof2     MS       F    p-unc    p-GG-corr     n2    eps
--------------------  -----  -------  -------  -----  ------  -------  -----------  -----  -----
LOC                   0.014        2       48  0.007   1.916    0.158        0.168  0.009  0.791
MOONEYCATEGORY        0.909        1       24  0.909  87.207    0.000        0.000  0.552  1.000
LOC * MOONEYCATEGORY  0.040        2       48  0.020   3.728    0.031        0.038  0.024  0.868



In [47]:
pg.pairwise_ttests(data=accPerSubLocs, dv='HOLISTIC', within=[ 'MOONEYCATEGORY', 'LOC'], subject='SUBINIT', marginal=True, padjust='holm', effsize= 'eta-square')

,Contrast,MOONEYCATEGORY,A,B,Paired,Parametric,T,dof,Tail,p-unc,p-corr,p-adjust,BF10,eta-square
0,MOONEYCATEGORY,-,high holistic,low holistic,True,True,9.338490,24.0,two-sided,1.838834e-09,NaN,NaN,6.295e+06,0.569997
1,LOC,-,left,right,True,True,2.144567,24.0,two-sided,4.232098e-02,0.126963,holm,1.462,0.033009
2,LOC,-,left,fovea,True,True,-0.163264,24.0,two-sided,8.716775e-01,0.871678,holm,0.213,0.000313
3,LOC,-,right,fovea,True,True,-1.774901,24.0,two-sided,8.860032e-02,0.177201,holm,0.823,0.023867
4,MOONEYCATEGORY * LOC,high holistic,left,right,True,True,1.091465,24.0,two-sided,2.859049e-01,0.571810,holm,0.36,0.008863
5,MOONEYCATEGORY * LOC,high holistic,left,fovea,True,True,-1.555923,24.0,two-sided,1.328160e-01,0.486420,holm,0.609,0.024279
6,MOONEYCATEGORY * LOC,high holistic,right,fovea,True,True,-2.335347,24.0,two-sided,2.821400e-02,0.169284,holm,2.026,0.057018
7,MOONEYCATEGORY * LOC,low holistic,left,right,True,True,1.688544,24.0,two-sided,1.042601e-01,0.486420,holm,0.728,0.040083
8,MOONEYCATEGORY * LOC,low holistic,left,fovea,True,True,1.725548,24.0,two-sided,9.728404e-02,0.486420,holm,0.767,0.069966
9,MOONEYCATEGORY * LOC,low holistic,right,fovea,True,True,0.758489,24.0,two-sided,4.555428e-01,0.571810,holm,0.274,0.009833


# Model with absolute eccentricity 

## Model holistic 

### ANOVA 

## Model upright 

### ANOVA

ECCENTRICITY can be 0, 2, 4, 6, 8, 10 or 12 where 0 represents the fovea

In [20]:
# Fits the model with the interaction term
# This will also automatically include the main effects for each factor
model = ols('ACCURACY ~ C(ECCENTRICITY)*C(MOONEYCATEGORY)', upright_data).fit()

# Seeing if the overall model is significant
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")
if model.f_pvalue < 0.05:
    print('The overall model is significant')

Overall model F( 13, 16236) =  45.080, p =  0.0000
The overall model is significant


In [21]:
rp.summary_cont(upright_data.groupby(['MOONEYCATEGORY', 'ECCENTRICITY']))['ACCURACY']

/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:151: RuntimeWarning: invalid value encountered in sqrt
  l_ci = lambda x: numpy.mean(x) - (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))
/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:154: RuntimeWarning: invalid value encountered in sqrt
  u_ci = lambda x: numpy.mean(x) + (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))


N    Mean        SD        SE  95% Conf.  \
MOONEYCATEGORY ECCENTRICITY                                                
high holistic  0              625  0.8512  0.356176  0.014247   0.823276   
               2             1250  0.8304  0.375431  0.010619   0.809587   
               4             1250  0.7888  0.408323  0.011549   0.766164   
               6             1250  0.7736  0.418668  0.011842   0.750390   
               8             1250  0.7608  0.426766  0.012071   0.737141   
               10            1250  0.7216  0.448391  0.012682   0.696742   
               12            1250  0.6976  0.459481  0.012996   0.672128   
low holistic   0              625  0.9184  0.273974  0.010959   0.896920   
               2             1250  0.9032  0.295804  0.008367   0.886801   
               4             1250  0.8976  0.303295  0.008578   0.880786   
               6             1250  0.8936  0.308472  0.008725   0.876499   
               8             1250  0.8888  0.314505  0.008896   0.871365   
               10            1250  0.8656  0.341218  0.009651   0.846684   
               12            1250  0.8776  0.327878  0.009274   0.859423   

                             Interval  
MOONEYCATEGORY ECCENTRICITY            
high holistic  0             0.879124  
               2             0.851213  
               4             0.811436  
               6             0.796810  
               8             0.784459  
               10            0.746458  
               12            0.723072  
low holistic   0             0.939880  
               2             0.919599  
               4             0.914414  
               6             0.910701  
               8             0.906235  
               10            0.884516  
               12            0.895777

In [22]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ACCURACY   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     45.08
Date:                Wed, 12 Aug 2020   Prob (F-statistic):          1.05e-114
Time:                        11:39:19   Log-Likelihood:                -6876.1
No. Observations:               16250   AIC:                         1.378e+04
Df Residuals:                   16236   BIC:                         1.389e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
Intercept                                                   0.8512      0.015     57.577      0.000       0.822       0.880
C(ECCENTRICITY)[T.2]                                       -0.0208      0.018     -1.149      0.251      -0.056       0.015
C(ECCENTRICITY)[T.4]                                       -0.0624      0.018     -3.446      0.001      -0.098      -0.027
C(ECCENTRICITY)[T.6]                                       -0.0776      0.018     -4.286      0.000      -0.113      -0.042
C(ECCENTRICITY)[T.8]                                       -0.0904      0.018     -4.993      0.000      -0.126      -0.055
C(ECCENTRICITY)[T.10]                                      -0.1296      0.018     -7.158      0.000      -0.165      -0.094
C(ECCENTRICITY)[T.12]                                      -0.1536      0.018     -8.483      0.000      -0.189      -0.118
C(MOONEYCATEGORY)[T.low holistic]                           0.0672      0.021      3.214      0.001       0.026       0.108
C(ECCENTRICITY)[T.2]:C(MOONEYCATEGORY)[T.low holistic]      0.0056      0.026      0.219      0.827      -0.045       0.056
C(ECCENTRICITY)[T.4]:C(MOONEYCATEGORY)[T.low holistic]      0.0416      0.026      1.625      0.104      -0.009       0.092
C(ECCENTRICITY)[T.6]:C(MOONEYCATEGORY)[T.low holistic]      0.0528      0.026      2.062      0.039       0.003       0.103
C(ECCENTRICITY)[T.8]:C(MOONEYCATEGORY)[T.low holistic]      0.0608      0.026      2.374      0.018       0.011       0.111
C(ECCENTRICITY)[T.10]:C(MOONEYCATEGORY)[T.low holistic]     0.0768      0.026      2.999      0.003       0.027       0.127
C(ECCENTRICITY)[T.12]:C(MOONEYCATEGORY)[T.low holistic]     0.1128      0.026      4.405      0.000       0.063       0.163
==============================================================================
Omnibus:                     4176.679   Durbin-Watson:                   1.851
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8189.417
Skew:                          -1.663   Prob(JB):                         0.00
Kurtosis:                       4.019   Cond. No.                         27.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
print('\n\nThe interaction is not significant p = 0.1')
res




The interaction is not significant p = 0.1


,sum_sq,df,F,PR(>F)
C(ECCENTRICITY),15.478338,6.0,18.885800,4.947660e-22
C(MOONEYCATEGORY),59.584985,1.0,436.213533,1.301375e-95
C(ECCENTRICITY):C(MOONEYCATEGORY),4.987015,6.0,6.084876,2.230638e-06
Residual,2217.771200,16236.0,NaN,NaN


In [24]:
print('We remove the interaction term because it is not significant')
# Fits the model
model2 = ols('ACCURACY ~ C(ECCENTRICITY)+ C(MOONEYCATEGORY)', upright_data).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

We remove the interaction term because it is not significant
Overall model F( 7, 16242) =  78.357, p =  0.0000


In [25]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ACCURACY   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     78.36
Date:                Wed, 12 Aug 2020   Prob (F-statistic):          2.54e-112
Time:                        11:39:19   Log-Likelihood:                -6894.3
No. Observations:               16250   AIC:                         1.380e+04
Df Residuals:                   16242   BIC:                         1.387e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.8242      0.011     75.909      0.000       0.803       0.846
C(ECCENTRICITY)[T.2]                 -0.0180      0.013     -1.405      0.160      -0.043       0.007
C(ECCENTRICITY)[T.4]                 -0.0416      0.013     -3.246      0.001      -0.067      -0.016
C(ECCENTRICITY)[T.6]                 -0.0512      0.013     -3.995      0.000      -0.076      -0.026
C(ECCENTRICITY)[T.8]                 -0.0600      0.013     -4.682      0.000      -0.085      -0.035
C(ECCENTRICITY)[T.10]                -0.0912      0.013     -7.117      0.000      -0.116      -0.066
C(ECCENTRICITY)[T.12]                -0.0972      0.013     -7.585      0.000      -0.122      -0.072
C(MOONEYCATEGORY)[T.low holistic]     0.1211      0.006     20.866      0.000       0.110       0.132
==============================================================================
Omnibus:                     4175.865   Durbin-Watson:                   1.851
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8187.285
Skew:                          -1.665   Prob(JB):                         0.00
Kurtosis:                       3.999   Cond. No.                         11.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(ECCENTRICITY),15.478338,6.0,18.850391,5.477658e-22
C(MOONEYCATEGORY),59.584985,1.0,435.395678,1.937741e-95
Residual,2222.758215,16242.0,NaN,NaN


In [27]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(ECCENTRICITY),15.478338,2.579723,6.0,18.850391,5.477658e-22,0.006736,0.006378
C(MOONEYCATEGORY),59.584985,59.584985,1.0,435.395678,1.937741e-95,0.025931,0.025870
Residual,2222.758215,0.136852,16242.0,NaN,NaN,NaN,NaN


### Post-hoc testing 

In [28]:
mc = statsmodels.stats.multicomp.MultiComparison(upright_data['ACCURACY'], upright_data['ECCENTRICITY'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     0      2   -0.018 0.7844 -0.0563  0.0203  False
     0      4  -0.0416  0.023 -0.0799 -0.0033   True
     0      6  -0.0512 0.0016 -0.0895 -0.0129   True
     0      8    -0.06  0.001 -0.0983 -0.0217   True
     0     10  -0.0912  0.001 -0.1295 -0.0529   True
     0     12  -0.0972  0.001 -0.1355 -0.0589   True
     2      4  -0.0236 0.2815 -0.0549  0.0077  False
     2      6  -0.0332 0.0289 -0.0645 -0.0019   True
     2      8   -0.042 0.0015 -0.0733 -0.0107   True
     2     10  -0.0732  0.001 -0.1045 -0.0419   True
     2     12  -0.0792  0.001 -0.1105 -0.0479   True
     4      6  -0.0096    0.9 -0.0409  0.0217  False
     4      8  -0.0184  0.579 -0.0497  0.0129  False
     4     10  -0.0496  0.001 -0.0809 -0.0183   True
     4     12  -0.0556  0.001 -0.0869 -0.0243   True
     6      8  -0.0088    0.9 -0.0401  0.0225 

## Model inverted 

### ANOVA 

In [29]:
# Fits the model with the interaction term
# This will also automatically include the main effects for each factor
model = ols('ACCURACY ~ C(ECCENTRICITY)*C(MOONEYCATEGORY)', inverted_data).fit()

# Seeing if the overall model is significant
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")
if model.f_pvalue < 0.05:
    print('The overall model is significant')

Overall model F( 13, 16236) =  110.709, p =  0.0000
The overall model is significant


In [30]:
rp.summary_cont(inverted_data.groupby(['MOONEYCATEGORY', 'ECCENTRICITY']))['ACCURACY']

/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:151: RuntimeWarning: invalid value encountered in sqrt
  l_ci = lambda x: numpy.mean(x) - (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))
/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:154: RuntimeWarning: invalid value encountered in sqrt
  u_ci = lambda x: numpy.mean(x) + (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))


N    Mean        SD        SE  95% Conf.  \
MOONEYCATEGORY ECCENTRICITY                                                
high holistic  0              625  0.5952  0.491246  0.019650   0.556686   
               2             1250  0.5944  0.491204  0.013893   0.567169   
               4             1250  0.5624  0.496290  0.014037   0.534887   
               6             1250  0.5576  0.496870  0.014054   0.530055   
               8             1250  0.5464  0.498042  0.014087   0.518790   
               10            1250  0.5392  0.498660  0.014104   0.511556   
               12            1250  0.5160  0.499944  0.014141   0.488285   
low holistic   0              625  0.8640  0.343063  0.013723   0.837104   
               2             1250  0.8504  0.356822  0.010092   0.830619   
               4             1250  0.8312  0.374725  0.010599   0.810426   
               6             1250  0.8064  0.395277  0.011180   0.784487   
               8             1250  0.8224  0.382329  0.010814   0.801205   
               10            1250  0.7776  0.416025  0.011767   0.754537   
               12            1250  0.7784  0.415490  0.011752   0.755366   

                             Interval  
MOONEYCATEGORY ECCENTRICITY            
high holistic  0             0.633714  
               2             0.621631  
               4             0.589913  
               6             0.585145  
               8             0.574010  
               10            0.566844  
               12            0.543715  
low holistic   0             0.890896  
               2             0.870181  
               4             0.851974  
               6             0.828313  
               8             0.843595  
               10            0.800663  
               12            0.801434

In [31]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ACCURACY   R-squared:                       0.081
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     110.7
Date:                Wed, 12 Aug 2020   Prob (F-statistic):          4.15e-287
Time:                        11:39:20   Log-Likelihood:                -9900.3
No. Observations:               16250   AIC:                         1.983e+04
Df Residuals:                   16236   BIC:                         1.994e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
Intercept                                                   0.5952      0.018     33.424      0.000       0.560       0.630
C(ECCENTRICITY)[T.2]                                       -0.0008      0.022     -0.037      0.971      -0.044       0.042
C(ECCENTRICITY)[T.4]                                       -0.0328      0.022     -1.504      0.133      -0.076       0.010
C(ECCENTRICITY)[T.6]                                       -0.0376      0.022     -1.724      0.085      -0.080       0.005
C(ECCENTRICITY)[T.8]                                       -0.0488      0.022     -2.238      0.025      -0.092      -0.006
C(ECCENTRICITY)[T.10]                                      -0.0560      0.022     -2.568      0.010      -0.099      -0.013
C(ECCENTRICITY)[T.12]                                      -0.0792      0.022     -3.631      0.000      -0.122      -0.036
C(MOONEYCATEGORY)[T.low holistic]                           0.2688      0.025     10.674      0.000       0.219       0.318
C(ECCENTRICITY)[T.2]:C(MOONEYCATEGORY)[T.low holistic]     -0.0128      0.031     -0.415      0.678      -0.073       0.048
C(ECCENTRICITY)[T.4]:C(MOONEYCATEGORY)[T.low holistic]  -2.411e-16      0.031  -7.82e-15      1.000      -0.060       0.060
C(ECCENTRICITY)[T.6]:C(MOONEYCATEGORY)[T.low holistic]     -0.0200      0.031     -0.648      0.517      -0.080       0.040
C(ECCENTRICITY)[T.8]:C(MOONEYCATEGORY)[T.low holistic]      0.0072      0.031      0.233      0.815      -0.053       0.068
C(ECCENTRICITY)[T.10]:C(MOONEYCATEGORY)[T.low holistic]    -0.0304      0.031     -0.986      0.324      -0.091       0.030
C(ECCENTRICITY)[T.12]:C(MOONEYCATEGORY)[T.low holistic]    -0.0064      0.031     -0.207      0.836      -0.067       0.054
==============================================================================
Omnibus:                     9892.852   Durbin-Watson:                   1.923
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2153.041
Skew:                          -0.685   Prob(JB):                         0.00
Kurtosis:                       1.859   Cond. No.                         27.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
print('\n\nThe interaction is not significant p = 0.8')
res



The interaction is not significant p = 0.8


,sum_sq,df,F,PR(>F)
C(ECCENTRICITY),11.691138,6.0,9.831450,7.565930e-11
C(MOONEYCATEGORY),272.937600,1.0,1377.131378,2.076662e-289
C(ECCENTRICITY):C(MOONEYCATEGORY),0.613600,6.0,0.515996,7.966973e-01
Residual,3217.859200,16236.0,NaN,NaN


In [33]:
print('We remove the interaction term because it is not significant')
# Fits the model
model2 = ols('ACCURACY ~ C(ECCENTRICITY)+ C(MOONEYCATEGORY)', upright_data).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

We remove the interaction term because it is not significant
Overall model F( 7, 16242) =  78.357, p =  0.0000


In [34]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ACCURACY   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     78.36
Date:                Wed, 12 Aug 2020   Prob (F-statistic):          2.54e-112
Time:                        11:39:20   Log-Likelihood:                -6894.3
No. Observations:               16250   AIC:                         1.380e+04
Df Residuals:                   16242   BIC:                         1.387e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.8242      0.011     75.909      0.000       0.803       0.846
C(ECCENTRICITY)[T.2]                 -0.0180      0.013     -1.405      0.160      -0.043       0.007
C(ECCENTRICITY)[T.4]                 -0.0416      0.013     -3.246      0.001      -0.067      -0.016
C(ECCENTRICITY)[T.6]                 -0.0512      0.013     -3.995      0.000      -0.076      -0.026
C(ECCENTRICITY)[T.8]                 -0.0600      0.013     -4.682      0.000      -0.085      -0.035
C(ECCENTRICITY)[T.10]                -0.0912      0.013     -7.117      0.000      -0.116      -0.066
C(ECCENTRICITY)[T.12]                -0.0972      0.013     -7.585      0.000      -0.122      -0.072
C(MOONEYCATEGORY)[T.low holistic]     0.1211      0.006     20.866      0.000       0.110       0.132
==============================================================================
Omnibus:                     4175.865   Durbin-Watson:                   1.851
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8187.285
Skew:                          -1.665   Prob(JB):                         0.00
Kurtosis:                       3.999   Cond. No.                         11.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(ECCENTRICITY),15.478338,6.0,18.850391,5.477658e-22
C(MOONEYCATEGORY),59.584985,1.0,435.395678,1.937741e-95
Residual,2222.758215,16242.0,NaN,NaN


In [36]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(ECCENTRICITY),15.478338,2.579723,6.0,18.850391,5.477658e-22,0.006736,0.006378
C(MOONEYCATEGORY),59.584985,59.584985,1.0,435.395678,1.937741e-95,0.025931,0.025870
Residual,2222.758215,0.136852,16242.0,NaN,NaN,NaN,NaN


### Post-hoc testing 

In [37]:
mc = statsmodels.stats.multicomp.MultiComparison(inverted_data['ACCURACY'], inverted_data['ECCENTRICITY'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     0      2  -0.0072    0.9 -0.0546  0.0402  False
     0      4  -0.0328 0.3895 -0.0802  0.0146  False
     0      6  -0.0476 0.0479  -0.095 -0.0002   True
     0      8  -0.0452  0.073 -0.0926  0.0022  False
     0     10  -0.0712  0.001 -0.1186 -0.0238   True
     0     12  -0.0824  0.001 -0.1298  -0.035   True
     2      4  -0.0256 0.4478 -0.0643  0.0131  False
     2      6  -0.0404 0.0338 -0.0791 -0.0017   True
     2      8   -0.038 0.0579 -0.0767  0.0007  False
     2     10   -0.064  0.001 -0.1027 -0.0253   True
     2     12  -0.0752  0.001 -0.1139 -0.0365   True
     4      6  -0.0148    0.9 -0.0535  0.0239  False
     4      8  -0.0124    0.9 -0.0511  0.0263  False
     4     10  -0.0384  0.053 -0.0771  0.0003  False
     4     12  -0.0496  0.003 -0.0883 -0.0109   True
     6      8   0.0024    0.9 -0.0363  0.0411 

# Model with absolute eccentricity 

## Model upright 

### ANOVA

ECCENTRICITY can be-12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10 or 12 where 0 represents the fovea

In [38]:
# Fits the model with the interaction term
# This will also automatically include the main effects for each factor
model = ols('ACCURACY ~ C(MOONEYCATEGORY)*C(LOC)', upright_data).fit()

# Seeing if the overall model is significant
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")
if model.f_pvalue < 0.05:
    print('The overall model is significant')
    
    
print(rp.summary_cont(upright_data.groupby(['MOONEYCATEGORY', 'LOC']))['ACCURACY'])
print(model.summary())


# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
print(res)
print('\n\nThe interaction is not significant p = 0.2')

Overall model F( 5, 16244) =  95.836, p =  0.0000
The overall model is significant


                         N      Mean        SD        SE  95% Conf.  Interval
MOONEYCATEGORY LOC                                                           
high holistic  fovea   625  0.851200  0.356176  0.014247   0.823276  0.879124
               left   3750  0.773867  0.418382  0.006832   0.760476  0.787258
               right  3750  0.750400  0.432839  0.007068   0.736546  0.764254
low holistic   fovea   625  0.918400  0.273974  0.010959   0.896920  0.939880
               left   3750  0.891467  0.311094  0.005080   0.881510  0.901424
               right  3750  0.884000  0.320268  0.005230   0.873749  0.894251
                            OLS Regression Results                            
Dep. Variable:               ACCURACY   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:  

/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:151: RuntimeWarning: invalid value encountered in sqrt
  l_ci = lambda x: numpy.mean(x) - (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))
/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:154: RuntimeWarning: invalid value encountered in sqrt
  u_ci = lambda x: numpy.mean(x) + (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))


In [39]:
# Fits the model with the interaction term
# This will also automatically include the main effects for each factor
model = ols('ACCURACY ~ C(MOONEYCATEGORY)*C(ECC_PLOT)', upright_data).fit()

# Seeing if the overall model is significant
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")
if model.f_pvalue < 0.05:
    print('The overall model is significant')
    
    
print(rp.summary_cont(upright_data.groupby(['MOONEYCATEGORY', 'ECC_PLOT']))['ACCURACY'])
print(model.summary())


# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
print(res)
print('\n\nThe interaction is not significant p = 0.1')

mc = statsmodels.stats.multicomp.MultiComparison(upright_data['ACCURACY'], upright_data['ECC_PLOT'])
mc_results = mc.tukeyhsd()
print(mc_results)

Overall model F( 25, 16224) =  23.932, p =  0.0000
The overall model is significant


                           N    Mean        SD        SE  95% Conf.  Interval
MOONEYCATEGORY ECC_PLOT                                                      
high holistic  -6.0      625  0.7216  0.448570  0.017943   0.686432  0.756768
               -5.0      625  0.7328  0.442852  0.017714   0.698080  0.767520
               -4.0      625  0.7632  0.425459  0.017018   0.729844  0.796556
               -3.0      625  0.7856  0.410734  0.016429   0.753398  0.817802
               -2.0      625  0.8032  0.397898  0.015916   0.772005  0.834395
               -1.0      625  0.8368  0.369844  0.014794   0.807804  0.865796
                0.0      625  0.8512  0.356176  0.014247   0.823276  0.879124
                1.0      625  0.8240  0.381125  0.015245   0.794120  0.853880
                2.0      625  0.7744  0.418312  0.016732   0.741604  0.807196
                3.0      625  0.7616  0.426446  0.017058

/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:151: RuntimeWarning: invalid value encountered in sqrt
  l_ci = lambda x: numpy.mean(x) - (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))
/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:154: RuntimeWarning: invalid value encountered in sqrt
  u_ci = lambda x: numpy.mean(x) + (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))


Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
  -6.0   -5.0   0.0072    0.9 -0.0425  0.0569  False
  -6.0   -4.0   0.0312 0.6535 -0.0185  0.0809  False
  -6.0   -3.0   0.0432 0.1669 -0.0065  0.0929  False
  -6.0   -2.0    0.056 0.0119  0.0063  0.1057   True
  -6.0   -1.0   0.0776  0.001  0.0279  0.1273   True
  -6.0    0.0    0.088  0.001  0.0383  0.1377   True
  -6.0    1.0   0.0624 0.0022  0.0127  0.1121   True
  -6.0    2.0   0.0368 0.4094 -0.0129  0.0865  False
  -6.0    3.0   0.0304 0.6873 -0.0193  0.0801  False
  -6.0    4.0   0.0248    0.9 -0.0249  0.0745  False
  -6.0    5.0  -0.0136    0.9 -0.0633  0.0361  False
  -6.0    6.0  -0.0184    0.9 -0.0681  0.0313  False
  -5.0   -4.0    0.024    0.9 -0.0257  0.0737  False
  -5.0   -3.0    0.036 0.4473 -0.0137  0.0857  False
  -5.0   -2.0   0.0488   0.06 -0.0009  0.0985  False
  -5.0   -1.0   0.0704  0.001  0.0207  0.1201 

You can correct for homoscedasticity by selecting hc3 under heteroscedasticity-consistent inference

In [40]:
# Fits the model with the interaction term
# This will also automatically include the main effects for each factor
model = ols('ACCURACY ~ C(MOONEYCATEGORY)*C(LOC)', upright_data).fit()

# Seeing if the overall model is significant
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")
if model.f_pvalue < 0.05:
    print('The overall model is significant')
    
    
print(rp.summary_cont(upright_data.groupby(['MOONEYCATEGORY', 'LOC']))['ACCURACY'])
print(model.summary())


# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2, robust = 'hc3')
print(res)
print('\n\nThe interaction is not significant p = 0.2')

Overall model F( 5, 16244) =  95.836, p =  0.0000
The overall model is significant


                         N      Mean        SD        SE  95% Conf.  Interval
MOONEYCATEGORY LOC                                                           
high holistic  fovea   625  0.851200  0.356176  0.014247   0.823276  0.879124
               left   3750  0.773867  0.418382  0.006832   0.760476  0.787258
               right  3750  0.750400  0.432839  0.007068   0.736546  0.764254
low holistic   fovea   625  0.918400  0.273974  0.010959   0.896920  0.939880
               left   3750  0.891467  0.311094  0.005080   0.881510  0.901424
               right  3750  0.884000  0.320268  0.005230   0.873749  0.894251
                            OLS Regression Results                            
Dep. Variable:               ACCURACY   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:  

/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:151: RuntimeWarning: invalid value encountered in sqrt
  l_ci = lambda x: numpy.mean(x) - (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))
/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:154: RuntimeWarning: invalid value encountered in sqrt
  u_ci = lambda x: numpy.mean(x) + (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))


                               sum_sq       df           F        PR(>F)
C(MOONEYCATEGORY)           58.318135      1.0  424.430178  4.094956e-93
C(LOC)                       5.115688      2.0   18.615584  8.406310e-09
C(MOONEYCATEGORY):C(LOC)     1.521390      2.0    5.536218  3.948847e-03
Residual                  2231.980267  16244.0         NaN           NaN


The interaction is not significant p = 0.2


## Model inverted

### ANOVA 

In [41]:
# Fits the model with the interaction term
# This will also automatically include the main effects for each factor
model = ols('ACCURACY ~C(LOC)*C(MOONEYCATEGORY)', inverted_data).fit()

# Seeing if the overall model is significant
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")
if model.f_pvalue < 0.05:
    print('The overall model is significant')
    
    
print(rp.summary_cont(inverted_data.groupby(['MOONEYCATEGORY', 'LOC']))['ACCURACY'])
print(model.summary())


# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
print(res)
print('\n\nThe interaction is not significant p = 0.7')

Overall model F( 5, 16244) =  277.859, p =  0.0000
The overall model is significant


                         N      Mean        SD        SE  95% Conf.  Interval
MOONEYCATEGORY LOC                                                           
high holistic  fovea   625  0.595200  0.491246  0.019650   0.556686  0.633714
               left   3750  0.555467  0.496980  0.008116   0.539560  0.571373
               right  3750  0.549867  0.497573  0.008125   0.533941  0.565792
low holistic   fovea   625  0.864000  0.343063  0.013723   0.837104  0.890896
               left   3750  0.804267  0.396817  0.006480   0.791566  0.816967
               right  3750  0.817867  0.386006  0.006303   0.805512  0.830221
                            OLS Regression Results                            
Dep. Variable:               ACCURACY   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic: 

/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:151: RuntimeWarning: invalid value encountered in sqrt
  l_ci = lambda x: numpy.mean(x) - (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))
/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:154: RuntimeWarning: invalid value encountered in sqrt
  u_ci = lambda x: numpy.mean(x) + (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))


In [42]:
# Fits the model with the interaction term
# This will also automatically include the main effects for each factor
model = ols('ACCURACY ~ C(MOONEYCATEGORY)*C(ECC_PLOT)', inverted_data).fit()

# Seeing if the overall model is significant
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")
if model.f_pvalue < 0.05:
    print('The overall model is significant')
    
    
print(rp.summary_cont(inverted_data.groupby(['MOONEYCATEGORY', 'ECC_PLOT']))['ACCURACY'])
print(model.summary())


# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
print(res)
print('\n\nThe interaction is not significant p = 0.9')

mc = statsmodels.stats.multicomp.MultiComparison(inverted_data['ACCURACY'], inverted_data['ECC_PLOT'])
mc_results = mc.tukeyhsd()
print(mc_results)

Overall model F( 25, 16224) =  58.154, p =  0.0000
The overall model is significant


/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:151: RuntimeWarning: invalid value encountered in sqrt
  l_ci = lambda x: numpy.mean(x) - (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))
/opt/anaconda3/lib/python3.7/site-packages/researchpy/summary.py:154: RuntimeWarning: invalid value encountered in sqrt
  u_ci = lambda x: numpy.mean(x) + (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))




                           N    Mean        SD        SE  95% Conf.  Interval
MOONEYCATEGORY ECC_PLOT                                                      
high holistic  -6.0      625  0.5344  0.499215  0.019969   0.495262  0.573538
               -5.0      625  0.5264  0.499702  0.019988   0.487223  0.565577
               -4.0      625  0.5568  0.497161  0.019886   0.517823  0.595777
               -3.0      625  0.5616  0.496588  0.019864   0.522667  0.600533
               -2.0      625  0.5488  0.498011  0.019920   0.509756  0.587844
               -1.0      625  0.6048  0.489285  0.019571   0.566440  0.643160
                0.0      625  0.5952  0.491246  0.019650   0.556686  0.633714
                1.0      625  0.5840  0.493288  0.019732   0.545326  0.622674
                2.0      625  0.5760  0.494586  0.019783   0.537224  0.614776
                3.0      625  0.5536  0.497517  0.019901   0.514595  0.592605
                4.0      625  0.5360  0.499102  0.019964   0.4

# Repeated measures 

In [73]:
aov = pg.rm_anova(data=upright_data, dv='ACCURACY', within=['MOONEYCATEGORY', 'ECC_PLOT'], subject='SUBINIT', detailed=True, effsize='n2')
pg.print_table(aov)
pg.pairwise_ttests(data=upright_data, dv='ACCURACY', within=[ 'MOONEYCATEGORY', 'ECC_PLOT'], subject='SUBINIT', marginal=True, padjust='holm', effsize= 'eta-square')
# x[x['Contrast'] == 'MOONEYCATEGORY * ECC_PLOT_ABS'].to_csv('ttests_abseccs.csv')


ANOVA SUMMARY

Source                        SS    ddof1    ddof2     MS        F    p-unc    p-GG-corr     n2    eps
-------------------------  -----  -------  -------  -----  -------  -------  -----------  -----  -----
MOONEYCATEGORY             2.383        1       24  2.383  123.055    0.000        0.000  0.330  1.000
ECC_PLOT                   0.659       12      288  0.055    8.781    0.000        0.000  0.091  0.553
MOONEYCATEGORY * ECC_PLOT  0.227       12      288  0.019    3.241    0.000        0.003  0.031  0.572



,Contrast,MOONEYCATEGORY,A,B,Paired,Parametric,T,dof,Tail,p-unc,p-corr,p-adjust,BF10,eta-square
0,MOONEYCATEGORY,-,high holistic,low holistic,True,True,-11.093003,24.0,two-sided,6.245772e-11,NaN,NaN,1.519e+08,0.312470
1,ECC_PLOT,-,-6,2,True,True,-1.912521,24.0,two-sided,6.781611e-02,1.000000,holm,1.01,0.025625
2,ECC_PLOT,-,-6,1,True,True,-3.726275,24.0,two-sided,1.049044e-03,0.061894,holm,33.295,0.071005
3,ECC_PLOT,-,-6,5,True,True,0.931944,24.0,two-sided,3.606464e-01,1.000000,holm,0.312,0.004274
4,ECC_PLOT,-,-6,0,True,True,-5.114896,24.0,two-sided,3.107722e-05,0.002238,holm,771.289,0.148519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,MOONEYCATEGORY * ECC_PLOT,low holistic,-2,-1,True,True,-0.618547,624.0,two-sided,5.364405e-01,1.000000,holm,0.054,0.000273
231,MOONEYCATEGORY * ECC_PLOT,low holistic,-2,3,True,True,0.599693,624.0,two-sided,5.489288e-01,1.000000,holm,0.054,0.000250
232,MOONEYCATEGORY * ECC_PLOT,low holistic,-4,-1,True,True,-1.177060,624.0,two-sided,2.396205e-01,1.000000,holm,0.09,0.001045
233,MOONEYCATEGORY * ECC_PLOT,low holistic,-4,3,True,True,0.000000,624.0,two-sided,1.000000e+00,1.000000,holm,0.045,0.000000


In [75]:
aov = pg.rm_anova(data=allsubjects, dv='ACCURACY', within=['COND', 'MOONEYCATEGORY', 'ECC_PLOT'], subject='SUBINIT', detailed=True, effsize='n2')
pg.print_table(aov)
pg.pairwise_ttests(data=allsubjects, dv='ACCURACY', within=['COND', 'MOONEYCATEGORY', 'ECC_PLOT'], subject='SUBINIT', marginal=True, padjust='holm', effsize= 'eta-square')
# x[x['Contrast'] == 'MOONEYCATEGORY * ECC_PLOT_ABS'].to_csv('ttests_abseccs.csv')

ValueError: Repeated measures ANOVA with three or more factors are not yet supported.

In [77]:
aovrm2way = AnovaRM(allsubjects, 'ACCURACY', 'SUBINIT', within=['COND','ECC_PLOT_ABS', 'MOONEYCATEGORY'], aggregate_func='mean')
res2way = aovrm2way.fit()

print(res2way)

                             Anova
                                 F Value  Num DF  Den DF  Pr > F
----------------------------------------------------------------
COND                             218.7806 1.0000  24.0000 0.0000
ECC_PLOT_ABS                      22.1966 6.0000 144.0000 0.0000
MOONEYCATEGORY                   207.6181 1.0000  24.0000 0.0000
COND:ECC_PLOT_ABS                  0.3024 6.0000 144.0000 0.9348
COND:MOONEYCATEGORY               68.5187 1.0000  24.0000 0.0000
ECC_PLOT_ABS:MOONEYCATEGORY        2.4708 6.0000 144.0000 0.0264
COND:ECC_PLOT_ABS:MOONEYCATEGORY   2.9421 6.0000 144.0000 0.0097



# Descriptive 

In [85]:
descriptives = allsubjects.groupby(['SUBINIT', 'COND', 'MOONEYCATEGORY'], as_index=False).mean()

In [87]:
stats.sem(descriptives[descriptives['COND'] == 'upright']['ACCURACY'])

0.01526104217555353

In [88]:
np.mean(descriptives[(descriptives['MOONEYCATEGORY'] == 'high holistic') & (descriptives['COND'] == 'inverted')]['ACCURACY'])

0.5559384615384615

In [89]:
stats.sem(descriptives[(descriptives['MOONEYCATEGORY'] == 'high holistic') & (descriptives['COND'] == 'inverted')]['ACCURACY'])

0.015677942142465165

In [90]:
np.mean(descriptives[(descriptives['MOONEYCATEGORY'] == 'low holistic') & (descriptives['COND'] == 'upright')]['ACCURACY'])

0.8900923076923078

In [91]:
stats.sem(descriptives[(descriptives['MOONEYCATEGORY'] == 'low holistic') & (descriptives['COND'] == 'upright')]['ACCURACY'])

0.019231401015263085

In [92]:
np.mean(descriptives[(descriptives['MOONEYCATEGORY'] == 'low holistic') & (descriptives['COND'] == 'inverted')]['ACCURACY'])

0.8151384615384617

In [93]:
stats.sem(descriptives[(descriptives['MOONEYCATEGORY'] == 'low holistic') & (descriptives['COND'] == 'inverted')]['ACCURACY'])

0.02142749436571853

In [95]:
np.mean(allsubjects[(allsubjects['MOONEYCATEGORY'] == 'high holistic') & (allsubjects['LOC'] == 'fovea') & (allsubjects['COND'] == 'upright')]['ACCURACY'])

0.8512

In [97]:
descriptives_2 = allsubjects.groupby(['SUBINIT', 'COND', 'MOONEYCATEGORY', 'LOC'], as_index=False).mean()

In [98]:
stats.sem(descriptives_2[(descriptives_2['MOONEYCATEGORY'] == 'high holistic') & (descriptives_2['LOC'] == 'fovea') & (descriptives_2['COND'] == 'upright')]['ACCURACY'])

0.02233323383062411